# Crawling_exercise2
# (Optional) 웹 크롤링2 - Dynamic Crawling
## 201823871 박지헌
## 0. 라이브러리

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time

In [5]:
import warnings; warnings.simplefilter('ignore')

## 1. Selenium 기초
* 자신의 크롬 버전을 확인하고 크롬 웹드라이버를 다운받아놓아야합니다.
> 2020.09.13 기준 최신 버전: 85.0.4183.102

### 1.1. Simple Text Crawling
멜론 사이트에서 노래 제목을 크롤링해보자
> URL: https://www.melon.com/chart/index.htm

In [15]:
DRIVER_PATH = 'chromedriver_win32/chromedriver.exe'

In [16]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) 
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# title crawling
title = WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > div")))


# print("Title: {}".format(title.text))

title.text

'STAY\nThe Kid LAROI, Justin Bieber'

In [17]:
# 2번째 제목 크롤링
WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.XPATH, "//*[@id='frm']/div/table/tbody/tr[1]/td[6]/div/div"))).text

'STAY\nThe Kid LAROI, Justin Bieber'

### 1.2. Text Crawling with for loop
위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자

In [18]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
for i in range(1, 51):
    title = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f"//*[@id='frm']/div/table/tbody/tr[{i}]/td[6]/div/div")))
    title_list.append(title.text)
    
print(title_list)

['STAY\nThe Kid LAROI, Justin Bieber', '신호등\n이무진', 'My Universe\nColdplay, 방탄소년단', 'OHAYO MY NIGHT\n디핵 (D-Hack), PATEKO (파테코)', 'Permission to Dance\n방탄소년단', 'Next Level\naespa', '바라만 본다\nMSG워너비(M.O.M)', 'Butter\n방탄소년단', 'Weekend\n태연 (TAEYEON)', '낙하 (with 아이유)\nAKMU (악뮤)', 'Queendom\nRed Velvet (레드벨벳)', 'Dynamite\n방탄소년단', '시간을 거슬러 (낮에 뜨는 달 X 케이윌)\n케이윌', '다정히 내 이름을 부르면\n경서예지, 전건호', '좋아좋아\n조정석', 'DUMB DUMB\n전소미', 'Bad Habits\nEd Sheeran', 'Peaches (Feat. Daniel Caesar & Giveon)\nJustin Bieber', '이제 나만 믿어요\n임영웅', '헤픈 우연\n헤이즈 (Heize)', '가을 타나 봐\n이무진', '고백\n멜로망스', 'Sticker\nNCT 127', '비와 당신\n이무진', '별빛 같은 나의 사랑아\n임영웅', '고백\n어반자카파', '그대라는 사치\n임영웅', 'HERO\n임영웅', 'Dun Dun Dance\n오마이걸 (OH MY GIRL)', '라일락\n아이유', '색안경 (STEREOTYPE)\nSTAYC(스테이씨)', '다시 사랑한다면 (김필 Ver.)\n임영웅', "롤린 (Rollin')\n브레이브걸스", 'ASAP\nSTAYC(스테이씨)', 'Savage Love (Laxed - Siren Beat) (BTS Remix)\nJawsh 685, Jason Derulo, 방탄소년단', 'Celebrity\n아이유', '비가 오는 날엔 (2021)\n헤이즈 (Heize)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)\n방탄소년단', '끝

### 1.3. Text Crawling (Click & Back)
클릭하고 나오기 -> 동적 크롤링 가능 (가사 크롤링 가능)

노래 제목에 링크가 걸려있기 때문에, 해당 링크까지의 XPath를 추가한다.

In [19]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[5]/div/a')))
click_element.click()    

# back
driver.back()


# 2번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[2]/td[5]/div/a')))
click_element.click()    

# back
driver.back()

### 1.4. Text Crawling including contents
* 1.2처럼 for문과 함께 써보자! (첫 페이지 5개의 글에 대해 title, artist, heart(하트 갯수), lyrics(가사)를 크롤링

* 1.3에서 사용한 click & back을 활용하자

In [20]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
artist_list = []
heart_list = []
lyrics_list = []

# crawling (TOP 5)
for i in range(1, 6):
    # click
    click_element = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{i}]/td[5]/div/a')))
    click_element.click()

    # title crawling
    title = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))
    title_list.append(title.text)

    # artist crawling
    artist = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
    artist_list.append(artist.text)
    
    # heart crawling
    heart = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
    heart_list.append(heart.text)

    # lyrics crawling
    lyrics = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
    lyrics_list.append(lyrics.text)
    
    # back
    driver.back()
    
print(title_list)
print(artist_list)
print(heart_list)
print(lyrics_list)

['STAY', '신호등', 'My Universe', 'OHAYO MY NIGHT', 'Permission to Dance']
['The Kid LAROI', '이무진', 'Coldplay', '디핵 (D-Hack)', '방탄소년단']
['153,841', '215,924', '75,833', '134,202', '176,827']
["I do the same thing I told you\nthat I never would\nI told you I'd change\neven when I knew I never could\nI know that I can't find\nnobody else\nas good as you\nI need you to stay\nneed you to stay hey Oh\nI get drunk wake up\nI'm wasted still\nI realize the time\nthat I wasted here\nI feel like you can't\nfeel the way I feel\nOh I'll be fucked up\nif you can't be right here\nOh ooh-woah\nOh ooh-woah ooh-woah\nOh ooh-woah\nOh ooh-woah ooh-woah\nOh ooh-woah\nOh ooh-woah ooh-woah\nOh I'll be fucked up\nif you can't be right here\nI do the same thing I told you\nthat I never would\nI told you I'd change\neven when I knew I never could\nI know that I can't find\nnobody else\nas good as you\nI need you to stay\nneed you to stay hey\nI do the same thing I told you\nthat I never would\nI told you I'd chan

In [21]:
# 결과 변수
raw_result = {'title': title_list,
              'artist': artist_list,
              'heart': heart_list,
              'lyrics': lyrics_list}

result = pd.DataFrame(raw_result)

# # csv 파일로 save
result.to_csv("MelonTop5", mode='w')

# driver 종료
driver.quit()

In [22]:
result

,title,artist,heart,lyrics
0,STAY,The Kid LAROI,"153,841",I do the same thing I told you\nthat I never w...
1,신호등,이무진,"215,924",이제야 목적지를 정했지만\n가려한 날 막아서네 난 갈 길이 먼데\n새빨간 얼굴로 화...
2,My Universe,Coldplay,"75,833","You, you are my universe and\nI just want to p..."
3,OHAYO MY NIGHT,디핵 (D-Hack),"134,202",너를 사랑하고 있어\n너를 사랑하고 있어\n자기야 날 사랑해주면 안 될까\n말처럼 ...
4,Permission to Dance,방탄소년단,"176,827",It’s the thought of being young\nWhen your hea...


## 2. Image Crawling
이미지 크롤링하기

* 1번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[1]/td[4]/div/a/img
* 2번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[2]/td[4]/div/a/img
...

* 50번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[50]/td[4]/div/a/img
현재는 가사 보기가 4번째 td로 옮겨짐

#### STEP1. URL Crawling

In [23]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
link_list = []

# # img crawling (TOP 50)
for i in range(1, 51):
    
    img = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(4) > div > a > img")))

    link_list.append(img.get_attribute('src'))

print(link_list)

['https://cdnimg.melon.co.kr/cm2/album/images/106/46/395/10646395_20210707141710_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/07/796/10607796_20210513201807_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/107/20/913/10720913_20210923173742_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/47/520/10447520_20200619123343_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/48/182/10648182_20210709104950_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/09/232/10609232_20210517155130_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/38/275/10638275_20210625172521_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quali

#### STEP2. Download images using URLs
자신의 디렉토리에 img 폴더 생성하고 실행

In [24]:
import os
import urllib.request

count = 0
os.makedirs("./img/", exist_ok=True)
for link in link_list:
    count += 1
    urllib.request.urlretrieve(link, './img/img' + str(count) + '.jpg')